In [1]:
pip install selenium == 4.17.2

91.91s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


zsh:1: = not found
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By # CSS 선택자를 지정하기 위해서 필요한 라이브러리
from selenium.webdriver.common.keys import Keys # 셀레니움 자동화를 위해 필요한 라이브러리
import time


# 셀레니움 우회
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=chrome_options)
# 이 코드가 상세페이지 접근을 가능하게 한 이유:
# 많은 웹사이트들은 봇이나 자동화된 브라우저를 감지하여 접근을 차단합니다.
# "--disable-blink-features=AutomationControlled" 옵션은 브라우저가 자동화되고 있다는 
# 흔적을 숨겨서 웹사이트가 셀레니움을 사용 중인 것을 감지하지 못하게 합니다.
# 이렇게 하면 웹사이트는 일반 사용자가 접속한 것처럼 인식하여 
# 상세페이지 접근을 허용하게 됩니다.

driver.get("https://bbn.kiwoom.com/bbs/VBbsBoardBCOSZView")

## 질문 글 제목만 추출하기

In [23]:
데이터들 = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')

for 데이터 in 데이터들:
    제목 = 데이터.find_element(By.CSS_SELECTOR, '.notice-title').text
    글번호 = 데이터.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text

    # 공지 제외
    # 질문 글 가져오기 (글 번호 있는 것만)
    if not '공지' in 제목 and 글번호 != '':
        
        # 제목 끝에 있는 N 제거 
        뉴아이콘 = 데이터.find_elements(By.CSS_SELECTOR, '.icon-new')
        
        # 만약 제목 끝에 N이 있으면, 제목 끝(-1)을 제거해주세요.
        if 뉴아이콘:
            제목 = 제목[:-1]
        # 그리고 '제거한 것 + 조건에 해당하지 않은 것'을 실행해주세요
        print(제목)

조건 검색시 기간 변경 방법
검색식문의입니다
조건검색식 문의
검색식 부탁드립니다
조건검색에서


## 각 글의 '제목 클릭> 내용 수집 > 뒤로가기' 반복 크롤링

In [ ]:
행개수 = len(driver.find_elements(By.CSS_SELECTOR, 'tbody > tr'))

for i in range(행개수):
    # 페이지 이동 후 DOM이 변경되므로 매번 요소를 다시 찾아야 함
    데이터들 = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    
    # 각 로우들의 데이터를 순서대로 확인하는 중요한 부분
    데이터 = 데이터들[i]
    # print(데이터)
    
    # 각 행에서 제목과 글번호를 매번 찾아야 함
    제목 = 데이터.find_element(By.CSS_SELECTOR, '.notice-title').text
    글번호 = 데이터.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text

    # 공지 제외
    # 질문 글 가져오기 (글 번호 있는 것만)
    if not '공지' in 제목 and 글번호 != '':
        
        # 제목 끝에 있는 N 제거 
        뉴아이콘 = 데이터.find_elements(By.CSS_SELECTOR, '.icon-new')
        
        # 만약 제목 끝에 N이 있으면, 제목 끝(-1)을 제거해주세요.
        if 뉴아이콘:
            제목 = 제목[:-1]
        # 그리고 '제거한 것 + 조건에 해당하지 않은 것'을 실행해주세요
        print(제목)

        # 질문 글에 있는 내용만 가져오려면?
            # 1. 질문 제목 글 클릭
            # 2. 글에 있는 본문 크롤링
            # 3. 뒤로 가기
            # 4. 1~3번 반복
        링크 = 데이터.find_element(By.CSS_SELECTOR, '.notice-title')
        링크.click()
        time.sleep(1)  # 페이지 로딩 시간 증가

        driver.back()
        time.sleep(1)  # 뒤로가기 후 페이지 로딩 시간 증가

In [ ]:
# 크롤링 하고자 하는 데이터가 요청 방식이 GET인지 POST인지 확인한다
# GET: URL에 데이터가 표시되어 있음 (예: https://example.com?name=value)
# POST: URL에 데이터가 표시되지 않고 요청 본문에 담겨 전송됨

# POST 요청을 할 때는 'Content-Type'을 확인해야 함
# 'Content-Type'은 서버에 어떤 형식으로 데이터를 보낼지 알려주는 역할을 함
# 주요 Content-Type:
# - application/x-www-form-urlencoded: 일반 폼 데이터 (data 인자 사용)
# - application/json: JSON 형식 데이터 (json 인자 사용)

import requests

# 키움증권 게시판의 Content-Type은 'application/x-www-form-urlencoded'임
# 따라서 requests.post()의 'data' 인자를 사용해야 함

# 서버에 보낼 데이터 준비
data = {
    "pageNo": 1,          # 페이지 번호
    "searchTp": 1,        # 검색 타입
    "pagePerBlock": 10,   # 한 페이지당 보여줄 게시글 수
    "_reqAgent": "ajax"   # 요청 방식
}

# POST 요청 보내기
response = requests.post("https://bbn.kiwoom.com/bbs/SBbsBoardBCOSZListAjax", data=data)

# 응답 확인 방법:
# 1. response.text: 텍스트 형태로 응답 내용 확인 (HTML이 나오면 추가 작업 필요)
# 2. response.json(): JSON 형태로 응답 내용 변환 (서버가 JSON을 반환할 때만 사용 가능)

# JSON 형태로 응답 데이터 받기
dataset = response.json()
dataset  # 결과 출력

## 패킷스니핑_POST_1페이지 데이터 가져오기

In [ ]:
# 서버에 보내는 요청 데이터 쉽게 추출하는 방법
# 1. 'Copy cURL'을 클릭해서 값을 복사한다.
# 2. 복사한 값을 'https://curlconverter.com/'에 붙여 넣는다.
# 3. 붙여 넣어서 나온 값을 그대로 복사 및 붙여넣기해서 응답 데이터를 받아본다.
# --------------------------------------------


import requests

cookies = {
    'WMONID': 'MuyIY1SvT8e',
    'JSESSIONID': '0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx',
    'evfw_PllKwu1': '54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

headers = {
    'Accept': '*/*, application/evfwlgIyr;q=0.1',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://bbn.kiwoom.com',
    'Referer': 'https://bbn.kiwoom.com/bbs/VBbsBoardBCOSZView',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'X-Reply-With': 'ajax',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    # 'Cookie': 'WMONID=MuyIY1SvT8e; JSESSIONID=0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx; evfw_PllKwu1=54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

data = {
    '_reqAgent': 'ajax',
}

response = requests.post('https://bbn.kiwoom.com/bbs/SBbsBoardWnameAjax', cookies=cookies, headers=headers, data=data)


cookies = {
    'WMONID': 'MuyIY1SvT8e',
    'JSESSIONID': '0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx',
    'evfw_PllKwu1': '54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

headers = {
    'Accept': '*/*, application/evfwlgIyr;q=0.1',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://bbn.kiwoom.com',
    'Referer': 'https://bbn.kiwoom.com/bbs/VBbsBoardBCOSZView',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'X-Reply-With': 'ajax',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    # 'Cookie': 'WMONID=MuyIY1SvT8e; JSESSIONID=0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx; evfw_PllKwu1=54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

data = {
    'pageNo': '1',
    'custId': '',
    'f_keyField': '',
    'f_key': '',
    'searchTp': '1',
    'pagePerBlock': '10',
    'maxGrpStep': '00000703249998',
    'minGrpStep': '00000701459996',
    'prevPageNo': '',
    '_reqAgent': 'ajax',
}

response = requests.post('https://bbn.kiwoom.com/bbs/SBbsBoardBCOSZListAjax', cookies=cookies, headers=headers, data=data)
dataset = response.json()
데이터들 = dataset['boardList']



for 데이터 in 데이터들:
    # print(데이터['ttl'])
    제목 = 데이터['ttl']
    날짜 = 데이터['makeTime']
    내용 = 데이터['dtlCntn']
    print(제목, 날짜, 내용, sep = '\n')
    print('='*20)

## 패킷스니핑_POST_10페이지 데이터까지 가져오기

In [ ]:
# Payload에서 페이지마다 변하는 키와 값을 눈 여겨본다.
# --------------------------------------------


import requests
import pandas as pd
import openpyxl

cookies = {
    'WMONID': 'MuyIY1SvT8e',
    'JSESSIONID': '0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx',
    'evfw_PllKwu1': '54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

headers = {
    'Accept': '*/*, application/evfwlgIyr;q=0.1',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://bbn.kiwoom.com',
    'Referer': 'https://bbn.kiwoom.com/bbs/VBbsBoardBCOSZView',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'X-Reply-With': 'ajax',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    # 'Cookie': 'WMONID=MuyIY1SvT8e; JSESSIONID=0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx; evfw_PllKwu1=54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

data = {
    '_reqAgent': 'ajax',
}

response = requests.post('https://bbn.kiwoom.com/bbs/SBbsBoardWnameAjax', cookies=cookies, headers=headers, data=data)


cookies = {
    'WMONID': 'MuyIY1SvT8e',
    'JSESSIONID': '0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx',
    'evfw_PllKwu1': '54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

headers = {
    'Accept': '*/*, application/evfwlgIyr;q=0.1',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://bbn.kiwoom.com',
    'Referer': 'https://bbn.kiwoom.com/bbs/VBbsBoardBCOSZView',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36',
    'X-Reply-With': 'ajax',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    # 'Cookie': 'WMONID=MuyIY1SvT8e; JSESSIONID=0D3nyolcacKcOVZgzeONUkZo1n6KrRBlYKVm0cNJItmODRxvvbUJZeIICsHnYZAa.a3dzX2RvbWFpbi9iYnMwMTAx; evfw_PllKwu1=54cb9db7bbaccc2ff8b1f9238966dd373c2732b4s',
}

result = []
for i in range(1, 11):
    data = {
        'pageNo': i,
        'custId': '',
        'f_keyField': '',
        'f_key': '',
        'searchTp': '1',
        'pagePerBlock': '10',
        'maxGrpStep': '00000703249998',
        'minGrpStep': '00000701459996',
        'prevPageNo': '',
        '_reqAgent': 'ajax',
    }

    response = requests.post('https://bbn.kiwoom.com/bbs/SBbsBoardBCOSZListAjax', cookies=cookies, headers=headers, data=data)
    dataset = response.json()
    데이터들 = dataset['boardList']


    for 데이터 in 데이터들:
        # print(데이터['ttl'])
        제목 = 데이터['ttl']
        날짜 = 데이터['makeTime']
        내용 = 데이터['dtlCntn']
        # print(제목, 날짜, 내용, sep = '\n')
        # print('='*20)
        result.append([제목, 날짜, 내용])

result    


df = pd.DataFrame(result, columns = ['제목', '날짜', '내용'])
df.head()